####Using Transformer LLMs for common tasks
1. Summarization
2. Translation
3. Question Answering
4. Table Question Answering
5. Fill Mask
6. Feature Extraction
7. Zero Shot Classification

In [0]:
from transformers import pipeline
import pandas as pd

2026-02-25 01:35:41.605162: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-25 01:35:41.616766: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-25 01:35:41.650619: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-25 01:35:41.665467: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-25 01:35:41.682739: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

####Translation
Translation is the task of converting text from one language to another.

Translate the given statement from english to hindi.\
You can use the Helsinki-NLP/opus-mt-en-hi model

In [0]:
statement = "This is a good book."
translator = pipeline(task="translation",model="Helsinki-NLP/opus-mt-en-hi")
result = translator(statement)


/databricks/python/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [0]:
print(result)

[{'translation_text': 'यह एक अच्छी किताब है.'}]


Translating to another Language (Telugu)

In [0]:
from transformers import pipeline

statement = "Humans are so cruel and selfish. No one cares about animals and the environment."

translator = pipeline(
    task="translation",
    model="facebook/nllb-200-distilled-600M",
    src_lang="eng_Latn",
    tgt_lang="tel_Telu"
)

result = translator(statement)

print(result[0]["translation_text"])

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


మానవులు చాలా క్రూరమైనవారు, స్వార్థపరులు. జంతువులు, పర్యావరణం ఎవరికీ పట్టించుకోవు.


####Question Answering
Question Answering models can retrieve the answer to a question from a given text, which is useful for searching for an answer in a document. Some question answering models can generate answers without context!

In [0]:
question = "Where do I live?"
context = "My name is ChatGpt, I became very popular now a days and I can answer your questions and I live in Mars."
qa_pipeline = pipeline(task="question-answering")


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [0]:
result = qa_pipeline(question = question, context = context)

In [0]:
print(result)

{'score': 0.9587178230285645, 'start': 99, 'end': 103, 'answer': 'Mars'}


In [0]:
context_ = """Tara and Sitara were two ducks. They were best friends. They lived in a beautiful pond.
Tara loved to eat small fishes, Tare really likes to eat fishes. Sitara loved swimming. They ate together. They
spent their whole day enjoying with each other.
"""

In [0]:
questions = ["What did Tara love?","Find out a describing word used for ‘pond’ in the story.","Where did Tara and Sitara live?"]
for ques_ in questions:
    result = qa_pipeline(question = ques_, context = context_)
    print(result)

{'score': 0.4285235106945038, 'start': 99, 'end': 118, 'answer': 'to eat small fishes'}
{'score': 0.09524255990982056, 'start': 72, 'end': 86, 'answer': 'beautiful pond'}
{'score': 0.5063747763633728, 'start': 70, 'end': 86, 'answer': 'a beautiful pond'}


####Table Question Answering
Table Question Answering (Table QA) is the answering a question about an information on a given table.

Generate the answer for the given question from the provided table.\
You can use the google/tapas-large-finetuned-wtq model

In [0]:
question = "how many apples available in the basket?"
data = {"fruits": ["grapes", "banana", "APPLES","Water melon"], "available_count": ["87", "53", "69","105"]}
table = pd.DataFrame.from_dict(data)

tqa_pipeline = pipeline(task="table-question-answering", model="google/tapas-large-finetuned-wtq")


Device set to use cpu


In [0]:
result = tqa_pipeline(table=table, query=question)

In [0]:
display(table)

fruits,available_count
grapes,87
banana,53
APPLES,69


In [0]:
print(result)


{'answer': 'SUM > 69', 'coordinates': [(2, 1)], 'cells': ['69'], 'aggregator': 'SUM'}


#### Querying out of the box

In [0]:
result_new = tqa_pipeline(table=table, query="Total number of muskmelon in the table ?")

In [0]:
result_new

{'answer': 'COUNT > grapes, banana, APPLES, Water melon',
 'coordinates': [(0, 0), (1, 0), (2, 0), (3, 0)],
 'cells': ['grapes', 'banana', 'APPLES', 'Water melon'],
 'aggregator': 'COUNT'}

####Fill-Mask
Masked language modeling is the task of masking some of the words in a sentence and predicting which words should replace those masks. These models are useful when we want to get a statistical understanding of the language in which the model is trained in.

Generate the sentance after filling the missing word.

In [0]:
sentance = "Humans are very <mask> to the animals.God please save them."
fm_pipeline = pipeline(task="fill-mask")
result = fm_pipeline(sentance)

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [0]:
display(result)

score,sequence,token,token_str
0.3535574972629547,Humans are very cruel to the animals.God please save them.,15939,cruel
0.09301217645406723,Humans are very sensitive to the animals.God please save them.,5685,sensitive
0.043063901364803314,Humans are very kind to the animals.God please save them.,761,kind
0.037907153367996216,Humans are very close to the animals.God please save them.,593,close
0.025424623861908913,Humans are very loyal to the animals.God please save them.,8259,loyal


####Feature Extraction
Feature extraction is the task of extracting features learnt in a model. These models can be used in RAG Approch.

Extract the features of the given text.\
You can use the facebook/bart-base model.

In [0]:
text = "Transformers is an awesome library!"
feature_extractor = pipeline(task="feature-extraction", model="facebook/bart-base")
result = feature_extractor(text, return_tensors = "pt")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [0]:
display(result)

tensor([[[ 2.5834,  2.7571,  0.9024,  ...,  1.5036, -0.0435, -0.8603],
         [-1.2850, -1.0094, -2.0826,  ...,  1.5993, -0.9017,  0.6426],
         [ 0.9082,  0.3896, -0.6843,  ...,  0.7061,  0.6517,  1.0550],
         ...,
         [ 0.6919, -1.1946,  0.2438,  ...,  1.3646, -1.8661, -0.1642],
         [-0.1701, -2.0019, -0.4223,  ...,  0.3680, -1.9704, -0.0068],
         [ 0.2520, -0.6869, -1.0582,  ...,  0.5198, -2.2106,  0.4547]]])

####Zero Shot Classification
Zero-shot text classification is a task in natural language processing where a model is trained on a set of labeled examples but is then able to classify new examples from previously unseen classes.

Classify the given sentance to the provided lables.\
You can use the facebook/bart-large-mnli model.

In [0]:
sentance = "I have a problem with my laptop that needs to be resolved asap!"
candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"]
zsc_pipiline = pipeline(task="zero-shot-classification", model="facebook/bart-large-mnli")
result = zsc_pipiline(sentance, candidate_labels = candidate_labels)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [0]:
display(result)

{'sequence': 'I have a problem with my laptop that needs to be resolved asap!',
 'labels': ['urgent', 'computer', 'not urgent', 'tablet', 'phone'],
 'scores': [0.571997344493866,
  0.4224330186843872,
  0.002626453759148717,
  0.0015330738388001919,
  0.0014100302942097187]}

In [0]:
sentence2 = "It is okay, it is acceptable to have 45 mins of delay."


In [0]:
candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"]
zsc_pipiline = pipeline(task="zero-shot-classification", model="facebook/bart-large-mnli")
result = zsc_pipiline(sentence2, candidate_labels = candidate_labels)

Device set to use cpu


In [0]:
result

{'sequence': 'It is okay, it is acceptable to have 45 mins of delay.',
 'labels': ['not urgent', 'tablet', 'computer', 'phone', 'urgent'],
 'scores': [0.8348758816719055,
  0.0617910772562027,
  0.04998967796564102,
  0.04736119508743286,
  0.005982189904898405]}